# Projet 4

In [1]:
# importation des librairies
import requests
from bs4 import BeautifulSoup

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

Exemple `URL_PAGE2` doit retourner :

<pre>{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
  'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
  'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
</pre>

In [2]:
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

def get_prices_from_url(url):
    # Initialisation de la liste des prix
    prices = {}
    
    # Requete
    r = requests.get(url)
    content = r.content
    # Récupération du contenu de la page
    soup = BeautifulSoup(content)
    
    # On récupère le contenu que l'on souhaite scrapper
    content_scrapping = soup.findAll('div', attrs={'class': f'pure-u-1 pure-u-md-1-{int(url[len(url)-6])+1}'})

    # On boucle sur chaque div
    for item in content_scrapping:
        new_dict = {}
        header = item.find('div', attrs = {'class': f'pricing-table-header'})
        
        # On récupère le type d'offre
        offer = header.find('h2').text
        
        # On récupère le prix
        prx = header.find('span', attrs = {'class': f'pricing-table-price'}).text.strip()
        new_dict['price'] = prx[0:len(prx)-10]
    
        # On récupère la taille du storage
        header1 = item.find('ul', attrs = {'class': 'pricing-table-list'}).findAll('li')
        stor = header1[3].text.strip().split()
        new_dict['storage'] = stor[0]
        
        # On récupère le nombre de databases
        datab = header1[4].text.strip().split()
        new_dict['databases'] = int(datab[0])
        
        # Mise à jour du dictionnaire
        prices[offer] = new_dict  
    
    return prices

p = get_prices_from_url(URL_PAGE3)

print(p)

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1}, 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5}, 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}, 'Privilege': {'price': '$99', 'storage': '1TB', 'databases': 100}}


Ecrire une fonction qui extrait des informations sur une bière de beowulf.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33 

La fonction doit retourner :
<pre>
{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}
</pre>

In [3]:
def extract_beer_infos(url):
    infos = {
        'name': None,
        'note': None,
        'price': None,
        'volume': None,
    }
     # Requete
    r = requests.get(url)
    content = r.content
    # Récupération du contenu de la page
    soup = BeautifulSoup(content)
    
    # On récupère le nom de la bière
    name = soup.find('title').text.split('|')
    infos['name'] = name[0]
    
    # On récupère la note
    score = soup.find('div', attrs={'class': "stars"})
    infos['note'] = score["data-percent"]
    
    # On récupère le prix
    infos['price'] = soup.find('span', attrs={'class': "price"}).text
    
    # On récupère le volume
    vol = soup.find('dd', attrs={'class': "small-6 medium-9 columns js-beer-volume"}).text.split()
    infos['volume'] = int(vol[0] )
    
    return infos
extract_beer_infos("https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33")

{'name': 'Mélusine Bio  ', 'note': '70', 'price': '38,99 €', 'volume': 33}

Cette URL retourne un JSON avec une liste de bières :

In [7]:
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"

Ecrire une fonction qui prend l'argument cet URL retourne les informations sur une liste de bière via l'API de beowulf.

Cette fonction doit retourner la liste des informations obtenues par la fonction `extract_beer_infos()` définie ci-dessus.

Chercher comment optimiser cette fonction en utilisant multiprocessing.Pool pour paralléliser les accès web.

Exemple de retour :

<pre>[{'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
    {'name': 'La Lager Sans Gluten de Vézelay', 'note': 60, 'price': 38.99, 'volume': 25},
    {'name': 'Brasserie De Sutter Brin de Folie', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'La Cristal IPA du Mont Blanc', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'La Parisienne Le Titi Parisien', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
    {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33},
    {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
    {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33}]
</pre>

In [8]:
def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    r = requests.get(url)
    # stock les données du JSON
    data = r.json()
    beer_pages = []
    # On boucle sur les éléments pour récupérer les pages
    for page in data['items']:
        beer_pages.append(url[:24] + page['contentReference'])
    # Sequential version (slow):
    beers = []
    # On boucle sur les différentes pages
    for link in beer_pages:
        beers.append(extract_beer_infos(link))

    # Parallel version (faster):
    
    return beers
extract_beer_list_infos(URL_BEERLIST_FRANCE)

[{'name': 'Gallia East IPA  ', 'note': '80', 'price': '42,99 €', 'volume': 33},
 {'name': 'La Lager Sans Gluten de Vézelay  ',
  'note': '60',
  'price': '38,99 €',
  'volume': 25},
 {'name': 'Brasserie De Sutter Brin de Folie  ',
  'note': '70',
  'price': '44,99 €',
  'volume': 33},
 {'name': 'La Cristal IPA du Mont Blanc  ',
  'note': '70',
  'price': '44,99 €',
  'volume': 33},
 {'name': 'Mélusine Bio  ', 'note': '70', 'price': '38,99 €', 'volume': 33},
 {'name': 'La Parisienne Le Titi Parisien  ',
  'note': '70',
  'price': '38,99 €',
  'volume': 33},
 {'name': 'Gallia Session IPA  ',
  'note': '70',
  'price': '42,99 €',
  'volume': 33},
 {'name': 'Ninkasi Brut IPA  ',
  'note': '70',
  'price': '44,99 €',
  'volume': 33},
 {'name': 'Pietra  ', 'note': '60', 'price': '38,99 €', 'volume': 33},
 {'name': 'Desperados  ', 'note': '60', 'price': '35,99 €', 'volume': 33},
 {'name': 'Gallia West IPA  ', 'note': '70', 'price': '42,99 €', 'volume': 33}]

In [9]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        # We should have 11 French beers:
        self.assertIsInstance(infos, list)
        self.assertEqual(len(infos), 11)
        # All of them are 25cl or 33cl:
        for beer in infos:
            self.assertIn(beer['volume'], [25, 33])

            
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [10]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Lesson4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Lesson4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 2.655s

OK
